# Chart Update Script

**MUST be stored in ROOT of chartstore repo (folder)**

In [15]:
# OPTIONAL - uncomment to use
# Update hongxiongmao from Github (only need if recent changes worth updating)
#! pip install git+https://github.com/djmcnay/hongxiongmao#egg=hongxiongmao --upgrade

In [2]:
import numpy as np
import pandas as pd
import plotly.offline as py

In [13]:
from hongxiongmao import pokemon
p = pokemon.pokemon_go()
p.download_and_run()
p.plotlyplot()
#py.plot(p.plot, filename='PlotlyHTMLexJS/pokemon.html', auto_open=False, include_plotlyjs=False)
py.plot(p.plot, filename='index.html', auto_open=False, include_plotlyjs=True)

'PlotlyHTMLexJS/pokemon.html'

## Push to Github Repo

In [17]:
! git add ./*                  # add all files to git staging area
! git commit -m "Updating Charts"
! git push origin master       # push to master branch

[master 0e5b3ae] Updating Charts
 1 file changed, 140 insertions(+), 160 deletions(-)
 rewrite update_charts.ipynb (79%)
Counting objects: 3, done.
Delta compression using up to 4 threads.
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 783 bytes | 783.00 KiB/s, done.
Total 3 (delta 2), reused 0 (delta 0)
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To https://github.com/djmcnay/pandachartstore.git
   336dcb6..0e5b3ae  master -> master
